In [1]:
!pip3 install torch==1.11 torchvision==0.12 torchaudio
!pip3 install opencv 
!pip3 install Pillow
!pip install easyocr

ERROR: Could not find a version that satisfies the requirement torch==1.11 (from versions: 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2)
ERROR: No matching distribution found for torch==1.11
ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv
     |████████████████████████████████| 70.8 MB 70.7 MB/s            
     |████████████████████████████████| 47.6 MB 100 kB/s             


In [2]:
import torchvision
import cv2
import torch
import numpy as np
import json
import pandas as pd
import easyocr

In [94]:
torchvision.__version__

'0.8.2'

In [95]:
#select device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [96]:
DEVICE

device(type='cuda')

In [97]:
with open("cocolabels.txt") as file: # Use file to refer to the file object
    CLASSES = file.read().splitlines()
with open('image_net_labels.json', 'r') as file:
    data=file.read()

IMGNETCLASSES = json.loads(data) 

In [98]:
#IMGNETCLASSES

In [99]:
#CLASSES


In [100]:
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [101]:
#object model
object_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(DEVICE)
object_model.eval()
#imagnet_model
#imagenet_model = torchvision.models.resnet101(pretrained=True).to(DEVICE)
#requires torchvision > 0.12
#bettter model than resnet
#imagenet_model = torchvision.models.efficientnet_b6(pretrained=True).to(DEVICE)
#imagenet_model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=True)
          (downsample)

In [102]:
imagenet_model = torchvision.models.resnet101(pretrained=True).to(DEVICE)
imagenet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [103]:
!pip install wget

In [104]:
import os
!pwd

/home/ec2-user/SageMaker/Final_Version


In [105]:
!rm -r 'basic'

In [106]:
!rm -r 'images'

In [107]:
!mkdir basic

In [108]:
!mkdir images

In [109]:
images=[]
conf=[]
lab=[]
media=[]

In [110]:
images.clear()
conf.clear()
lab.clear()

In [111]:
j=0

In [112]:
import os
import pandas as pd 
import wget

#read in data
data = pd.read_csv("merged_tweets_filtered.csv") 

# assuming you have a column named Column1 which contains the link, iterate #through and download
for index, row in data.iterrows():
    x=row['media_url']
    
    img = wget.download(x,out='basic')
    image = cv2.imread(img)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    orig = image.copy()
    image = image.transpose((2, 0, 1))
# add the batch dimension, scale the raw pixel intensities to the
# range [0, 1], and convert the image to a floating point tensor
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    image = torch.FloatTensor(image)
# send the input to the device and pass the it through the network to
# get the detections and predictions
    image = image.to(DEVICE)
    detections = object_model(image)[0]
    confidence =detections["scores"]

    for i in range(0, len(detections["boxes"])):
    # extract the confidence (i.e., probability) associated with
    # the prediction
        confidence = detections["scores"][i]
        label=detections['labels'][i] 
            
        
    # filter out weak detections by ensuring the confidence is
    # greater than the minimum confidence
        if confidence > .5 and label==9:
            
            x=confidence.cpu().detach().numpy()
            y=label.cpu().detach().numpy()
            #print(x)
            
            #print(float(confidence))
           # print(x)
            
            
        # extract the index of the class label from the
        # detections, then compute the (x, y)-coordinates of
        # the bounding box for the object
            idx = int(detections["labels"][i])
            box = detections["boxes"][i].detach().cpu().numpy()
            (startX, startY, endX, endY) = box.astype("int")
        # draw the bounding box and label on the frame
           # label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
           # print(label)
           # cv2.rectangle(orig, (startX, startY), (endX, endY),
            #COLORS[idx], 2)
            
           # y = startY - 15 if startY - 15 > 15 else startY + 15
            #cv2.putText(orig, label, (startX, y),
            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
            #cv2.imwrite('sampleimage.png',orig)
            #img = cv2.imread(orig)
            z=box.astype("int")
            list=z.tolist()
           
            cropped_image = orig[list[1]:list[3],list[0]:list[2]]
            
            j=j+1
            path="images/Cropped Image"+str(j)+".jpg"

            cv2.imwrite(path, cropped_image)
            images.append(path)
            conf.append(float(confidence))
            lab.append(idx)
            media.append(row['media_url'])
            print(path)
           
            
    

images/Cropped Image1.jpg
images/Cropped Image2.jpg
images/Cropped Image3.jpg
images/Cropped Image4.jpg
images/Cropped Image5.jpg
images/Cropped Image6.jpg
images/Cropped Image7.jpg
images/Cropped Image8.jpg
images/Cropped Image9.jpg
images/Cropped Image10.jpg
images/Cropped Image11.jpg
images/Cropped Image12.jpg
images/Cropped Image13.jpg
images/Cropped Image14.jpg
images/Cropped Image15.jpg
images/Cropped Image16.jpg
images/Cropped Image17.jpg
images/Cropped Image18.jpg
images/Cropped Image19.jpg
images/Cropped Image20.jpg
images/Cropped Image21.jpg
images/Cropped Image22.jpg
images/Cropped Image23.jpg
images/Cropped Image24.jpg
images/Cropped Image25.jpg
images/Cropped Image26.jpg
images/Cropped Image27.jpg
images/Cropped Image28.jpg
images/Cropped Image29.jpg
images/Cropped Image30.jpg
images/Cropped Image31.jpg
images/Cropped Image32.jpg
images/Cropped Image33.jpg
images/Cropped Image34.jpg
images/Cropped Image35.jpg
images/Cropped Image36.jpg
images/Cropped Image37.jpg
images/Cro

In [113]:
import pandas as pd
imag = pd.DataFrame(images,columns=['images'])
conf=pd.DataFrame(conf,columns=['confidence'])
lab=pd.DataFrame(lab,columns=['label'])
med=pd.DataFrame(media,columns=['med'])

In [114]:
med

,med
0,https://pbs.twimg.com/media/FOu6bsWWQAY3HpF.jpg
1,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg
2,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg
3,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg
4,https://pbs.twimg.com/media/FOdzWUcUcAALYM4.jpg
...,...
11343,https://pbs.twimg.com/media/FLsIXWpX0AU2iy4.jpg
11344,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg
11345,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg
11346,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg


In [115]:
result1=pd.concat([med,imag,conf,lab],axis=1)

In [116]:
result1

,med,images,confidence,label
0,https://pbs.twimg.com/media/FOu6bsWWQAY3HpF.jpg,images/Cropped Image1.jpg,0.997274,9
1,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg,images/Cropped Image2.jpg,0.888261,9
2,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg,images/Cropped Image3.jpg,0.870261,9
3,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg,images/Cropped Image4.jpg,0.551909,9
4,https://pbs.twimg.com/media/FOdzWUcUcAALYM4.jpg,images/Cropped Image5.jpg,0.997695,9
...,...,...,...,...
11343,https://pbs.twimg.com/media/FLsIXWpX0AU2iy4.jpg,images/Cropped Image11344.jpg,0.526897,9
11344,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg,images/Cropped Image11345.jpg,0.989100,9
11345,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg,images/Cropped Image11346.jpg,0.986886,9
11346,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg,images/Cropped Image11347.jpg,0.913547,9


In [117]:
from torchvision import transforms
#imagenet expects this formating
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [118]:
clas=[]
conf1=[]
img1=[]

In [119]:
clas.clear()
conf1.clear()
img1.clear()

In [120]:
from PIL import Image
import glob

In [121]:
folder_dir = 'images'
for img in sorted(glob.iglob(f'{folder_dir}/*'), key=os.path.getmtime):
    img1.append(img)
    print(img)
    img = Image.open(img)
    #img.show()
    img_t = transform(img).to(DEVICE)
    batch_t = torch.unsqueeze(img_t, 0)
    imagenet_det = imagenet_model(batch_t)
    probs = torch.softmax(imagenet_det,dim=1).detach().cpu().numpy()[0]
    top1 = probs.argsort()[::-1][:1]
    x=[(IMGNETCLASSES[i],probs[i]) for i in top1]
    a=x[0][0]
    b=x[0][1]
    clas.append(a)
    conf1.append(b)


images/Cropped Image1.jpg
images/Cropped Image2.jpg
images/Cropped Image3.jpg
images/Cropped Image4.jpg
images/Cropped Image5.jpg
images/Cropped Image6.jpg
images/Cropped Image7.jpg
images/Cropped Image8.jpg
images/Cropped Image10.jpg
images/Cropped Image9.jpg
images/Cropped Image11.jpg
images/Cropped Image12.jpg
images/Cropped Image13.jpg
images/Cropped Image14.jpg
images/Cropped Image15.jpg
images/Cropped Image16.jpg
images/Cropped Image17.jpg
images/Cropped Image18.jpg
images/Cropped Image19.jpg
images/Cropped Image20.jpg
images/Cropped Image23.jpg
images/Cropped Image21.jpg
images/Cropped Image22.jpg
images/Cropped Image24.jpg
images/Cropped Image25.jpg
images/Cropped Image27.jpg
images/Cropped Image26.jpg
images/Cropped Image28.jpg
images/Cropped Image29.jpg
images/Cropped Image30.jpg
images/Cropped Image31.jpg
images/Cropped Image32.jpg
images/Cropped Image33.jpg
images/Cropped Image34.jpg
images/Cropped Image36.jpg
images/Cropped Image35.jpg
images/Cropped Image37.jpg
images/Cro

In [122]:
#clas

In [123]:
#img1

In [124]:
#conf1

In [125]:
image2 = pd.DataFrame(img1,columns=['images'])
confidence1=pd.DataFrame(conf1,columns=['confidence2'])

class1=pd.DataFrame(clas,columns=['Class'])

In [126]:
class1

,Class
0,container ship
1,milk can
2,container ship
3,container ship
4,container ship
...,...
11343,aircraft carrier
11344,grey whale
11345,beaker
11346,polar bear


In [127]:
image2

,images
0,images/Cropped Image1.jpg
1,images/Cropped Image2.jpg
2,images/Cropped Image3.jpg
3,images/Cropped Image4.jpg
4,images/Cropped Image5.jpg
...,...
11343,images/Cropped Image11343.jpg
11344,images/Cropped Image11346.jpg
11345,images/Cropped Image11345.jpg
11346,images/Cropped Image11347.jpg


In [128]:
result2=pd.concat([image2,class1,confidence1],axis=1)
result2

,images,Class,confidence2
0,images/Cropped Image1.jpg,container ship,0.991774
1,images/Cropped Image2.jpg,milk can,0.296007
2,images/Cropped Image3.jpg,container ship,0.944609
3,images/Cropped Image4.jpg,container ship,0.403636
4,images/Cropped Image5.jpg,container ship,0.999592
...,...,...,...
11343,images/Cropped Image11343.jpg,aircraft carrier,0.988338
11344,images/Cropped Image11346.jpg,grey whale,0.408739
11345,images/Cropped Image11345.jpg,beaker,0.345697
11346,images/Cropped Image11347.jpg,polar bear,0.117711


In [129]:

# using merge function by setting how='left'
output2 = pd.merge(result1, result2, 
                   on='images', 
                   how='left')
  
# displaying result


In [130]:
output2

,med,images,confidence,label,Class,confidence2
0,https://pbs.twimg.com/media/FOu6bsWWQAY3HpF.jpg,images/Cropped Image1.jpg,0.997274,9,container ship,0.991774
1,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg,images/Cropped Image2.jpg,0.888261,9,milk can,0.296007
2,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg,images/Cropped Image3.jpg,0.870261,9,container ship,0.944609
3,https://pbs.twimg.com/media/FOjS1FAXsAMl-cl.jpg,images/Cropped Image4.jpg,0.551909,9,container ship,0.403636
4,https://pbs.twimg.com/media/FOdzWUcUcAALYM4.jpg,images/Cropped Image5.jpg,0.997695,9,container ship,0.999592
...,...,...,...,...,...,...
11343,https://pbs.twimg.com/media/FLsIXWpX0AU2iy4.jpg,images/Cropped Image11344.jpg,0.526897,9,cleaver,0.140875
11344,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg,images/Cropped Image11345.jpg,0.989100,9,beaker,0.345697
11345,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg,images/Cropped Image11346.jpg,0.986886,9,grey whale,0.408739
11346,https://pbs.twimg.com/media/FLmWSDaWQAAPJIQ.jpg,images/Cropped Image11347.jpg,0.913547,9,polar bear,0.117711


In [131]:
output2.to_csv('Final_results.csv', encoding='utf-8')

In [7]:
reader = easyocr.Reader(["en"], gpu = True)

text = []
#coordinates= []
image= []
confidence= []


In [8]:
import os

In [9]:
text.clear()
image.clear()
confidence.clear()

In [10]:
from PIL import Image
import glob
folder_dir = 'images'
for img in sorted(glob.iglob(f'{folder_dir}/*'), key=os.path.getmtime):
    images = cv2.imread(img)
    bounds = reader.readtext(images, paragraph=False)
    for bound in bounds:
        if bound[2]>0.1:
            text.append(bound[1])
            confidence.append(bound[2])
            #coordinates.append(bound[0])
            image.append(img)
            print(img)
    
    

images/Cropped Image7.jpg
images/Cropped Image9.jpg
images/Cropped Image11.jpg
images/Cropped Image12.jpg
images/Cropped Image15.jpg
images/Cropped Image29.jpg
images/Cropped Image30.jpg
images/Cropped Image32.jpg
images/Cropped Image33.jpg
images/Cropped Image35.jpg
images/Cropped Image38.jpg
images/Cropped Image39.jpg
images/Cropped Image41.jpg
images/Cropped Image41.jpg
images/Cropped Image42.jpg
images/Cropped Image43.jpg
images/Cropped Image44.jpg
images/Cropped Image45.jpg
images/Cropped Image47.jpg
images/Cropped Image54.jpg
images/Cropped Image57.jpg
images/Cropped Image58.jpg
images/Cropped Image73.jpg
images/Cropped Image75.jpg
images/Cropped Image76.jpg
images/Cropped Image89.jpg
images/Cropped Image93.jpg
images/Cropped Image93.jpg
images/Cropped Image97.jpg
images/Cropped Image100.jpg
images/Cropped Image101.jpg
images/Cropped Image104.jpg
images/Cropped Image116.jpg
images/Cropped Image117.jpg
images/Cropped Image119.jpg
images/Cropped Image120.jpg
images/Cropped Image125

In [11]:
!pwd

/home/ec2-user/SageMaker/Final_Version


In [152]:
reader = easyocr.Reader(["en"], gpu = True)

In [12]:
image3 = pd.DataFrame(image,columns=['images'])
confidence3=pd.DataFrame(confidence,columns=['confidenceOCR'])

Text=pd.DataFrame(text,columns=['Text'])

In [13]:
ocr=pd.concat([image3,Text,confidence3],axis=1)
ocr

,images,Text,confidenceOCR
0,images/Cropped Image7.jpg,Mi0z,0.588968
1,images/Cropped Image9.jpg,MI07,0.749628
2,images/Cropped Image11.jpg,Mi07,0.357847
3,images/Cropped Image12.jpg,150,0.528480
4,images/Cropped Image15.jpg,150,0.733686
...,...,...,...
6532,images/Cropped Image11329.jpg,GVAAD,0.488487
6533,images/Cropped Image11330.jpg,US,0.364088
6534,images/Cropped Image11330.jpg,'COAST,0.654002
6535,images/Cropped Image11330.jpg,GUAAD,0.429517


In [14]:
ocr.to_csv('ocr.csv', encoding='utf-8')